# Create EEG Sentences with Next-Char Probabilities


In [1]:
import random
import torch.nn.functional as F
from bundle.DataCraft import * 
from bundle.ApiCraft  import * 


# --- Configuration ---
model_path = "../../model/api/char_predictor.pth"           # Input: Pretrained NLP model path
characters_eeg_filepath = "../../data/characters_eeg.pkl"
sentences_filepath = "../../data/sentences_val.txt"             # Input: Sentences file
output_filepath = "../../data/sentences_eeg_val_set2.pkl"            # Output: Combined data

# NL Predictor Model Definition 


In [2]:
# Load characters from file
all_chars = load_characters()
char2idx = {ch: idx for idx, ch in enumerate(all_chars)}
idx2char = {idx: ch for ch, idx in char2idx.items()}
vocab_size = len(all_chars)

print("Vocabulary size:", vocab_size)
print("Characters in vocabulary:", all_chars)


def predict_next_chars(model, sentence, top_k=None):
 
    model.eval()
    with torch.no_grad():
        # Convert to indices, skipping unknown characters
        input_seq = [char2idx[ch] for ch in sentence if ch in char2idx]
        if not input_seq:
            print("Warning: Input sentence contains no known characters. Using empty sequence.")
            # Return uniform distribution if no valid input
            result = {ch: 1.0/vocab_size for ch in all_chars}
            return result

        input_seq = torch.tensor(input_seq).unsqueeze(0)
        output = model(input_seq)
        probs = F.softmax(output, dim=-1).squeeze(0)
        
        # If top_k is specified, get only top k predictions
        if top_k is not None:
            top_k = min(top_k, vocab_size)  # Ensure top_k doesn't exceed vocab size
            top_probs, top_indices = torch.topk(probs, top_k)
            
            result = {}
            for prob, idx in zip(top_probs, top_indices):
                result[idx2char[idx.item()]] = round(prob.item(), 4)
        else:
            # Return all probabilities
            result = {}
            for idx, prob in enumerate(probs):
                result[idx2char[idx]] = round(prob.item(), 4)
        
        return result

Loaded 37 characters from ../../data/characters.txt (including added space)
Vocabulary size: 37
Characters in vocabulary: ABCDEFGHIJKLMNOPQRSTUVWXYZ123456789_ 


# Add next-char prediction probabilities

In [3]:
# Load Sample Groups for characters
sample_groups_data = load_characters_eeg(characters_eeg_filepath)
if not sample_groups_data:
    exit()

# Load NLP Model
nlp_model = load_nlp_model(vocab_size,model_path)
if not nlp_model:
    exit()
    
all_chars = list(load_characters())
char_vector_len = len(all_chars)


# Read Sentences
sentences = []
print(f"Reading sentences from: {sentences_filepath}")
try:
    with open(sentences_filepath, "r") as f:
        sentences = [line.strip() for line in f if line.strip()] # Read non-empty lines
    print(f"Read {len(sentences)} sentences.")
except FileNotFoundError:
    print(f"Error: Sentences file not found at {sentences_filepath}")
    exit()
except Exception as e:
    print(f"Error reading sentences file: {e}")
    exit()

if not sentences:
    print("No sentences found in the file. Exiting.")
    exit()

# Process Sentences and Combine Data
processed_data = []
print("\nProcessing sentences to combine EEG chunks and probabilities...")

for sentence_idx, sentence in enumerate(sentences):
    print(f"Processing sentence {sentence_idx + 1}/{len(sentences)}:{sentence}")
    for char_idx, char in enumerate(sentence):
        prefix = sentence[:char_idx]
        
        if char==' ':
            continue
        
        # Check if character is valid and has data in set1
        if char in sample_groups_data and "set2" in sample_groups_data[char] and sample_groups_data[char]["set2"]:
            
            # Randomly select a chunk from set1 for this character
            available_chunks = sample_groups_data[char]["set2"]
            selected_chunk = random.choice(available_chunks)
            # Predict next character probabilities based on the prefix
            if len(prefix)!=0:
                next_char_probs = predict_next_chars(nlp_model, prefix)
            else:
                next_char_probs= {ch: 0 for ch in all_chars}  # Uniform distribution if no prefix
         
         
            prob_vector = np.array([next_char_probs.get(c, 0.0) for c in all_chars])  # shape (37,)
            # Repeat and truncate to get 78 values
            repeats = (78 + char_vector_len - 1) // char_vector_len  # ceiling division
            prob_vector_repeated = np.tile(prob_vector, repeats)[:78]  # shape (78,)
            # Expand to 78×64 matrix
            prob_matrix = np.tile(prob_vector_repeated[:, np.newaxis], (1, 64))  # shape (78, 64)
            
            combined_chunk = selected_chunk + [prob_matrix] 
            
            # Store the combined information
            char_data = {
                	"character": char,
                    "sentence": sentence,
                	"char_idx_in_sentence": char_idx,
                	"eeg_chunk": combined_chunk, 
                    "prob_matrix_78x64": prob_matrix
            }
            processed_data.append(char_data)
            
        else:
            # Handle cases where character is not in groups or set1 is missing/empty
            # Also handles characters not in our defined vocabulary (like punctuation if any)
            if char not in sample_groups_data:
                 print(f"  - Warning: Character {char} not found in sample groups. Skipping.")
            elif "set1" not in sample_groups_data[char] or not sample_groups_data[char]["set1"]:
                 print(f"  - Warning: No chunks found in set1 for character {char}. Skipping.")
            # else: # Character might be punctuation or not in NLP vocab
            #     print(f"  - Info: Skipping character 	"{char}	" (possibly not in vocab or no EEG data).")
            pass # Silently skip characters without EEG data or not in vocab


for i in range(0, len(processed_data)):
    if len(processed_data[i]["eeg_chunk"]) != 21:
        print(f"Warning: EEG chunk for character '{processed_data[i]['character']}' in sentence '{processed_data[i]['sentence']}' has unexpected length {len(processed_data[i]['eeg_chunk'])}. Expected 31.")
    

print(f"\nFinished processing. Collected data for {len(processed_data)} character instances.")

Attempting to load sample groups from: ../../data/characters_eeg.pkl
Successfully loaded sample groups dictionary.
Model loaded from ../../model/api/char_predictor.pth.
Loaded 37 characters from ../../data/characters.txt (including added space)
Reading sentences from: ../../data/sentences_val.txt
Read 201 sentences.

Processing sentences to combine EEG chunks and probabilities...
Processing sentence 1/201:A FINAL WAVE HIT LAND
Processing sentence 2/201:ONE EYE WATCHED THEM MOVE
Processing sentence 3/201:NEW FLAME LIT THE HOLE
Processing sentence 4/201:A THICK FACE LOOKED BACK
Processing sentence 5/201:ORANGE FLAME TORE THROUGH FIELD
Processing sentence 6/201:A THIN LINE GLOWED BRIGHT
Processing sentence 7/201:WIND LIFTED HER THICK HAIR
Processing sentence 8/201:THE CROW FLEW OVER PEAK
Processing sentence 9/201:ALL WENT QUIET AFTER NIGHT
Processing sentence 10/201:OLD MARK FELL OFF FENCE
Processing sentence 11/201:THEY DREW A YELLOW LINE
Processing sentence 12/201:HIDE THE MAP UNDER ROC

# Save Processed Data

In [4]:
print(f"\nSaving combined data to: {output_filepath}")
try:
    with open(output_filepath, "wb") as f:
        pickle.dump(processed_data, f)
    print(f"Successfully saved combined data to {output_filepath}.")
except Exception as e:
    print(f"Error saving combined data: {e}")


Saving combined data to: ../../data/sentences_eeg_val_set2.pkl
Successfully saved combined data to ../../data/sentences_eeg_val_set2.pkl.


# Load Processed Data

In [5]:
final_data = load_sentence_eeg_prob_data(output_filepath)

if final_data:
    # Print example of converted item
    print(f"Loaded {len(final_data)} items.")
    if final_data:
        print("Example of first item:", final_data[1]["character"])
        print("Example of first item:", final_data[1]["char_idx_in_sentence"])
        print("Example of first item:", final_data[1]["sentence"])
        print("Example of first item:", final_data[1]["eeg_chunk"][0:1])
        print("Example of first item:", final_data[1]["prob_matrix_78x64"].shape)
        print("Example of first item:", final_data[1]["prob_matrix_78x64"][0][0])
        print("Example of first item:", final_data[1]["prob_matrix_78x64"][7][0])
else:
    print("Failed to load data. Please check the file path.")

Attempting to load processed data from: ../../data/sentences_eeg_val_set2.pkl
Successfully loaded processed data.
Loaded 4286 items.
Example of first item: F
Example of first item: 2
Example of first item: A FINAL WAVE HIT LAND
Example of first item: [array([[1.3059361 , 2.691928  , 1.5563794 , ..., 1.6617911 , 0.7815503 ,
        2.036563  ],
       [0.9740189 , 2.009328  , 1.2256639 , ..., 1.0955331 , 0.4530468 ,
        1.2932538 ],
       [0.7131406 , 1.3396595 , 0.8999807 , ..., 0.52004457, 0.1002709 ,
        0.4957163 ],
       ...,
       [0.48322558, 1.1540679 , 1.7158124 , ..., 1.0644215 , 2.0420544 ,
        0.46619776],
       [0.43487802, 1.3251594 , 1.6341099 , ..., 0.7596864 , 2.0747058 ,
        0.27557445],
       [0.37294388, 1.3763406 , 1.4411949 , ..., 0.3765416 , 2.0643313 ,
        0.0278351 ]], dtype=float32)]
Example of first item: (78, 64)
Example of first item: 0.2747
Example of first item: 0.0667
